In [0]:
!git clone https://github.com/acmilannesta/Adult_readmission

Cloning into 'Adult_readmission'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 52 (delta 25), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (52/52), done.


In [0]:
!pip install tsfresh
!pip install pandas==0.22

In [0]:
import pandas as pd
import  numpy as np
from tsfresh import extract_relevant_features, extract_features

In [0]:
lab = pd.read_csv('Adult_readmission/PA_0501.csv', parse_dates=['SPECIMEN_COLLECT_DT', 'TRANSPLANT_DT', 'TRANSPLANT_DISCHARGE_DT'])
cond1 = (lab['TRANSPLANT_DISCHARGE_DT'] >= lab['SPECIMEN_COLLECT_DT'])
cond2 = ((lab['TRANSPLANT_DT'] - lab['SPECIMEN_COLLECT_DT']).map(lambda x: x.days) <= 365)
cond3 = (lab['TRANSPLANT_DT'] <= lab['SPECIMEN_COLLECT_DT'])
# lab = lab[cond1 & cond2]
y = pd.read_sas('Adult_readmission/combined_vars_04092019.sas7bdat')[['PERSON_ID', 'CODE_REHOSP']]

**Biomarker list created**

"markerlist": biomarkers with multiple time points

'singlemarker': biomarkers with the last time point taken before transplantation

In [0]:
markerlist = ['CREATININE',
           'GLUCOSE',
           'MAGNESIUM',
           'CHLORIDE',
           'ANION_GAP__BLOOD',
           'ALKALINE_PHOSPHATASE',
           'ALBUMIN',
           'BUN__BLOOD_UREA_NITROGEN_',
           'CO2',
           'HEMATOCRIT',
           'HEMOGLOBIN',
           'POTASSIUM_PLASMA',
           'SODIUM_PLASMA',
           'UREA_CREATININE_RATIO',
             ]
singlemarker = ['DRUG_SCREEN', 
                'GLUCOSE_URINE_QUALITATIVE',
                'NITRITE_URINE_QUALITATIVE', 
                'PROTEIN_URINE_QUALITATIVE',
                'HBA1C__HEMOGLOBIN_A1C_',
                'INDIRECT_BILIRUBIN',
                'NON_HDL_CHOLESTEROL',
                'PROGRAF__TACROLIMUS_',
                'PT_RESULT',
                'URIC_ACID',
               ]

In [0]:
biomarker_extract = y['PERSON_ID'].to_frame()
for marker in markerlist:
  if marker in ['CREATININE', 'BUN__BLOOD_UREA_NITROGEN_', 'UREA_CREATININE_RATIO']:
    biomarker = lab[cond1 & cond3][['PERSON_ID', 'SPECIMEN_COLLECT_DT', marker]].dropna().sort_values(['PERSON_ID', 'SPECIMEN_COLLECT_DT'])
  else:
    biomarker = lab[cond1 & cond2][['PERSON_ID', 'SPECIMEN_COLLECT_DT', marker]].dropna().sort_values(['PERSON_ID', 'SPECIMEN_COLLECT_DT'])
  biomarker['time'] = biomarker.groupby('PERSON_ID').cumcount()
  y_biomarker = biomarker['PERSON_ID'].drop_duplicates().to_frame().merge(y, on='PERSON_ID', how='left')
  y_biomarker.index=y_biomarker['PERSON_ID']  
  f = extract_relevant_features(biomarker.drop('SPECIMEN_COLLECT_DT', 1), y_biomarker['CODE_REHOSP'], column_id='PERSON_ID', column_sort='time')
  f['PERSON_ID'] = f.index
  biomarker_extract = biomarker_extract.merge(f, on='PERSON_ID', how='left')

In [0]:
for marker in singlemarker:
    tmp = lab[cond1][['PERSON_ID', marker]].dropna()
    assert(tmp.drop_duplicates().shape[0] == tmp.shape[0])
    biomarker_extract = biomarker_extract.merge(tmp, on='PERSON_ID', how='left')

In [0]:
singlemarker.extend(['PERSON_ID'])
feature_retain = list(biomarker_extract.drop(singlemarker, 1).corrwith(y['CODE_REHOSP']).sort_values().iloc[np.r_[0:50, -50:0]].index)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
singlemarker.extend(feature_retain)
biomarker_extract[singlemarker].to_csv('drive/My Drive/Adult readmission/feature_extracted_365days.csv', index=False)